<a href="https://colab.research.google.com/github/prav2909/Machine-Learning/blob/master/K_Nearest_Neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

k-nearest neighbors

Only used the already imported libraries `numpy` and `matplotlib.pyplot` 

In [0]:
# Load required packages and dataset. Do not modify.
import matplotlib.pyplot as plt
import numpy as np


def load_iris_dataset():
    from sklearn import datasets
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    return X, y
    
X, y = load_iris_dataset()

In [0]:
class_names = ['setosa', 'versicolor', 'virginica']
feature_names = ['sepal length', 'sepal width', 'petal length', 'petal width']

print(f'#samples: {X.shape[0]}')

for class_name, class_count in zip(class_names, np.bincount(y)):
    print(f'- class {class_name}: {class_count}')
print()
    
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
for feature_idx, feature_name in enumerate(feature_names):
    print(f'{feature_name}:')
    print(f'- mean: {mean[feature_idx]:.2f}')
    print(f'- std: {std[feature_idx]:.2f}')
    print()

3) Visualizing the variables Sepal length and Petal length in a scatter plot (Sepal length on the x-axis, petal length on the y-axis). Coloring each point of the plot according to its class.

In [0]:
fig, ax = plt.subplots(1, 1)
for class_idx, class_name in enumerate(class_names):
    ax.scatter(X[y == class_idx, 0], X[y == class_idx, 2], label=class_name)
ax.set_xlabel('sepal length [cm]')
ax.set_ylabel('petal length [cm]')
ax.legend();

4) Spliting the dataset randomly into training and test data. 70% of data should be used for training and 30% should be used for testing.

In [0]:
def train_test_split(X, y):
    """
    Returns X_train, X_test, y_train, y_test, 
        where X_train and X_test are the input features of the training and test set,
        and y_train and y_test are the class labels of the training and test set.
    """
    np.random.seed(2020)  # Ensure that the random split always returns the same result.
    
    threshold = int(0.7*X.shape[0])
    rnd_idx = np.random.permutation(X.shape[0])
    
    X_train = X[rnd_idx[:threshold]]
    X_test = X[rnd_idx[threshold:]]
    y_train = y[rnd_idx[:threshold]]
    y_test = y[rnd_idx[threshold:]]
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y)

assert (X_train.shape[0] + X_test.shape[0]) == X.shape[0]
assert (y_train.shape[0] + y_test.shape[0]) == y.shape[0]
assert X_train.shape[1] == X_test.shape[1]

In [0]:
X_min = np.min(X_train, axis=0)
X_max = np.max(X_train, axis=0)

def min_max_scale(X):
    return (X - X_min) / (X_max - X_min)

X_train = min_max_scale(X_train)
X_test = min_max_scale(X_test)

In [0]:
class KNearestNeighbors(object):
    def __init__(self, k, distance_metric='uniform'):
        self.k = k
        self.distance_metric = distance_metric
        
    def fit(self, X, y):
        """
        This functions saves the training data to be used during the prediction.
        """
        self.X = X
        self.y = y
    
    def predict(self, X):
        """
        Returns a vector of shape (n,) if X has shape (n,d), 
        where n is the number of samples and d is the number of features.
        """
        if self.distance_metric == 'uniform':
            dist = euclidean_distance
        else:
            dist = self.distance_metric
            
        # Compute pairwise distances between inputs and training samples.
        pw_distances = np.array([[dist(x1, x2) for x1 in X] for x2 in self.X])
        # Identify k nearest neighbors for each input sample.
        neighbors = np.argsort(pw_distances, axis=0)[:self.k].T
        
        labels = []
        for idx, nb in enumerate(neighbors):
            candidate_labels = np.unique(y_train[nb])
            candidate_weights = []

            for candidate in candidate_labels:
                if self.distance_metric == 'uniform': # uniform weights
                    candidate_weight = np.sum(self.y[nb] == candidate)
                else: # distance-weighted
                    weights = 1.0/pw_distances[nb, idx][self.y[nb] == candidate]
                    # If distance is 0, the corresponding weight will be infinity,
                    # which results in us just looking up the correct label or performing 1-NN.
                    candidate_weight = np.sum(weights)
                
                candidate_weights.append(candidate_weight)
            
            # Select the class label with highest weight.
            label_idx = np.argmax(candidate_weights)
            label = candidate_labels[label_idx]
            labels.append(label)
        
        return np.array(labels)

    
def euclidean_distance(x1, x2):
    """
    Given vectors x1 and x2 with shape (n,) returns distance between vectors as float.
    """
    return np.sqrt(np.sum((x1 - x2)*(x1 - x2)))


In [0]:
def precision(y_pred, y_true):
    # We compute the macro-average
    labels = set(y_pred).union(set(y_true))
    precisions = list()
    for label in labels:
        tp, fp, _, _ = tp_fp_tn_fn(y_pred, y_true, label)
        if (tp + fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        precisions.append(prec)
    return np.mean(precisions)


def recall(y_pred, y_true):
    # We compute the macro-average
    labels = set(y_pred).union(set(y_true))
    recalls = list()
    for label in labels:
        tp, _, _, fn = tp_fp_tn_fn(y_pred, y_true, label)
        recalls.append(tp / (tp + fn))
    return np.mean(recalls)


def f1score(y_pred, y_true):
    # We compute the macro-average
    labels = set(y_pred).union(set(y_true))
    f1scores = list()
    for label in labels:
        tp, fp, _, fn = tp_fp_tn_fn(y_pred, y_true, label)
        if (tp + fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        rec = tp / (tp + fn)
        f1scores.append(2 * (prec * rec) / (prec + rec))
    return np.mean(f1scores)


def tp_fp_tn_fn(y_pred, y_true, positive_label):
    # Count true positive, false positives, true negatives, false negatives for a specific class.
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for yp, yt in zip(y_pred, y_true):
        if yp == yt and yt == positive_label: # tp
            tp += 1
        elif yp != yt and yt == positive_label: # fp
            fp += 1
        elif yp == yt and yt != positive_label: # tn
            tn += 1
        else: # fn
            fn += 1
    return tp, fp, tn, fn
        

In [0]:
ks = (1, 3, 5)
metrics = ('uniform', 'euclidean')
hyperparameters = [(k, metric) for metric in metrics for k in ks]

scores = []
for k, metric in hyperparameters:
        classifier = KNearestNeighbors(
            k=k, 
            distance_metric='uniform' if metric == 'uniform' else euclidean_distance
        )
        classifier.fit(X_train, y_train)
        yhat_train = classifier.predict(X_train)
        yhat_test = classifier.predict(X_test)
                
        scores.append([
            [fnc(yhat, y) for fnc in (precision, recall, f1score)] 
            for yhat, y in ((yhat_train, y_train), (yhat_test, y_test))
        ])
        
scores = np.array(scores)
ks = np.array(ks)
fig, ax = plt.subplots(3, 2, figsize=(8, 8), sharex=True, sharey=True)
for row in range(3):
    for col in range(2):
        ax[row, col].bar(ks-0.25, scores[:3, col, row], width=0.5, label='Uniform')
        ax[row, col].bar(ks+0.25, scores[3:, col, row], width=0.5, label='Euclidian')
        ax[row, col].set_ylim((0.8, 1.0))
        
ax[0, 0].set_ylabel('Precision')
ax[1, 0].set_ylabel('Recall')
ax[2, 0].set_ylabel('F1-Score')
ax[0, 0].set_title('Training')
ax[0, 1].set_title('Test')
ax[-1, 0].set_xlabel('k')
ax[-1, 1].set_xlabel('k')
ax[-1, 0].set_xticks(ks)
ax[0, 1].legend();